This script aims to aggregate the reviews and sales rank metrics on a larger time frame.

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime, date, time, timedelta
from tqdm import tqdm

## Read Datasets

In [2]:
month = ["March", "April", "May", "June", "July", "August", "September", "October", "November"]
avery_sales_rank = pd.DataFrame()
for i in range(0,len(month)):
    sr_file_name = "SalesRankExport_Avery_" + month[i] + "2019.csv"
    df_bin = pd.read_csv(sr_file_name)
    avery_sales_rank = pd.concat([avery_sales_rank, df_bin], ignore_index = True)

/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (17,18,20,21,23,24,26,27,29,30,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [3]:
avery_review = pd.read_csv('Avery_reviews.csv')

/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (2,3,8,11,12,24,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
avery_price = pd.read_csv("BuyboxExport_eaf8d027-48c3-40c6-8147-7e41eded6c5c.csv")

/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1,21,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
avery_price.shape

(1682361, 28)

In [6]:
avery_sales_rank.columns

Index(['category_id1', 'category_id10', 'category_id2', 'category_id3',
       'category_id4', 'category_id5', 'category_id6', 'category_id7',
       'category_id8', 'category_id9', 'category_name1', 'category_name10',
       'category_name2', 'category_name3', 'category_name4', 'category_name5',
       'category_name6', 'category_name7', 'category_name8', 'category_name9',
       'category_rank1', 'category_rank10', 'category_rank2', 'category_rank3',
       'category_rank4', 'category_rank5', 'category_rank6', 'category_rank7',
       'category_rank8', 'category_rank9', 'date', 'end_ts', 'id', 'source',
       'start_ts'],
      dtype='object')

In [7]:
avery_sales_rank.head()

,category_id1,category_id10,category_id2,category_id3,category_id4,category_id5,category_id6,category_id7,category_id8,category_id9,...,category_rank5,category_rank6,category_rank7,category_rank8,category_rank9,date,end_ts,id,source,start_ts
0,office-products/1069410,NaN,office-products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019-03-11 19:34:59,2019-03-31,B00004Z5QO,amazon,2019-03-01
1,office-products/1069410,NaN,office-products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019-03-12 19:49:09,2019-03-31,B00004Z5QO,amazon,2019-03-01
2,office-products/1069410,NaN,office-products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019-03-13 20:08:07,2019-03-31,B00004Z5QO,amazon,2019-03-01
3,office-products/1069410,NaN,office-products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019-03-14 20:52:36,2019-03-31,B00004Z5QO,amazon,2019-03-01
4,office-products/1069410,NaN,office-products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019-03-15 19:37:16,2019-03-31,B00004Z5QO,amazon,2019-03-01


In [8]:
avery_price.columns

Index(['Source', 'id', 'name', 'rangestart', 'rangeend', 'buyboxtime', 'price',
       'high', 'low', 'stock', 'merchant', 'merchant_id', 'prime', 'subscribe',
       'pantry', 'addon', 'min', 'max', 'mapviolation', 'sellers', 'BU_custom',
       'MPN_custom', 'Brand_custom', 'Benchmark_custom', 'UPC',
       '100 PL_custom', 'Sprint_custom', 'SMU_custom'],
      dtype='object')

## Select Product Category

In [14]:
avery_sales_rank[avery_sales_rank['category_id1'] == 'office-products/490930011']['category_name1'].drop_duplicates()

609    Binder Index Dividers
610          Office Products
Name: category_name1, dtype: object

In [149]:
avery_sales_rank.groupby(by='category_id1', as_index=False).agg({'id': pd.Series.nunique}).sort_values(by = 'id', ascending=False).head(30)

,category_id1,id
65,office-products/1069410,371
92,office-products/1069636,338
147,office-products/490930011,278
82,office-products/1069588,244
60,office-products/1069392,190
50,office-products,186
81,office-products/1069584,173
58,office-products/1069388,137
79,office-products/1069574,133
59,office-products/1069390,108


In [142]:
# dataset1 = avery_sales_rank[avery_sales_rank['category_id1'] == 'office-products/1069410'][['id', 'date', 'category_rank1']]

In [143]:
# dataset1.to_csv("category1.csv")

In [150]:
# dataset2 = avery_sales_rank[avery_sales_rank['category_id1'] == 'industrial/8553235011']

In [151]:
# dataset2.to_csv("category2.csv")

In [12]:
products = avery_sales_rank[avery_sales_rank['category_id1'] == 'office-products/490930011']['id'].drop_duplicates()

In [13]:
products

609        B00006IBVA
630        B00006IBVG
693        B00006IBYA
1512       B001HA2H58
2079       B01FKQBAES
              ...    
5330623    B07J9WZD4G
5330893    B07NXX773Q
5330964    B07NZZLD19
7452001    B00GRKGBJQ
7579321    B01CGGOLWI
Name: id, Length: 278, dtype: object

In [15]:
avery_sales_rank_filtered = avery_sales_rank[avery_sales_rank['id'].isin(products)].set_index('id',drop=True)
avery_review_filtered = avery_review[avery_review['product'].isin(products)].set_index('product',drop=True)
# avery_price_filtered = avery_price[avery_price['id'].isin(products)].set_index('id',drop=True)

In [16]:
avery_price_filtered = avery_price[avery_price['id'].isin(products)].set_index('id',drop=True)

In [17]:
# create sales rank dictionary
avery_sales_rank_dict = {}
for i in avery_sales_rank_filtered.index.unique():
    avery_sales_rank_dict[i] = avery_sales_rank_filtered.loc[i, ['source','date', 'category_id1',
       'category_name1', 'category_rank1', 'category_id2', 'category_name2', 'category_rank2']]

In [18]:
# create review dictionary
avery_review_dict = {}
for i in avery_review_filtered.index.unique():
    avery_review_dict[i] = avery_review_filtered.loc[i, ['source','date','author', 'verified', 'vine', 'stars', 'pvotes', 'tvotes',
       'title', 'text', 'image', 'video']]

In [19]:
# create price dictionary
avery_price_dict = {}
for i in avery_price_filtered.index.unique():
    avery_price_dict[i] = avery_price_filtered.loc[i, ['id','buyboxtime','price',
       'high', 'low', 'stock', 'merchant']]

/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [20]:
# sales rank dictionary
# deal with duplicated values and time
for i in avery_sales_rank_dict:
    if type(avery_sales_rank_dict[i]) != pd.core.series.Series:
        avery_sales_rank_dict[i] = avery_sales_rank_dict[i].sort_values('date').reset_index()
        avery_sales_rank_dict[i].drop_duplicates(inplace = True)
        date_time = pd.to_datetime(avery_sales_rank_dict[i]['date'])
        a = avery_sales_rank_dict[i].index[0]
        avery_sales_rank_dict[i].drop(a, inplace = True)
        avery_sales_rank_dict[i]['month'] = date_time.dt.strftime('%Y-%m')
        avery_sales_rank_dict[i]['month'] = pd.to_datetime(avery_sales_rank_dict[i]['month'])
        avery_sales_rank_dict[i]['new_date'] = date_time.dt.strftime('%Y-%m-%d')
        avery_sales_rank_dict[i]['new_date'] = pd.to_datetime(avery_sales_rank_dict[i]['new_date'])

In [21]:
# reset index of sales rank dictionary
for i in avery_sales_rank_dict:
    if type(avery_sales_rank_dict[i]) != pd.core.series.Series:
        avery_sales_rank_dict[i] = avery_sales_rank_dict[i].reset_index(drop = True)

In [22]:
# reviews dictionary
# deal with duplicated values and time
for i in avery_review_dict:
    if type(avery_review_dict[i]) != pd.core.series.Series:
        avery_review_dict[i] = avery_review_dict[i].sort_values('date').reset_index()
        date_time = pd.to_datetime(avery_review_dict[i]['date'])
        avery_review_dict[i]['month'] = date_time.dt.strftime('%Y-%m')
        avery_review_dict[i]['month'] = pd.to_datetime(avery_review_dict[i]['month'])
        avery_review_dict[i]['new_date'] = date_time.dt.strftime('%Y-%m-%d')
        avery_review_dict[i]['new_date'] = pd.to_datetime(avery_review_dict[i]['new_date'])

In [23]:
# deal with time of price dictionary
for i in avery_price_dict:
    if type(avery_price_dict[i]) != pd.core.series.Series:
        avery_price_dict[i] = avery_price_dict[i].drop(labels = 'id',axis = 1)
        avery_price_dict[i] = avery_price_dict[i].sort_values('buyboxtime').reset_index()
        avery_price_dict[i].drop_duplicates(inplace = True)
        date_time = pd.to_datetime(avery_price_dict[i]['buyboxtime'])
        avery_price_dict[i]['month'] = date_time.dt.strftime('%Y-%m')
        avery_price_dict[i]['month'] = pd.to_datetime(avery_price_dict[i]['month']) 

In [24]:
avery_sales_rank_dict['B00006IBVA']

,id,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date
0,B00006IBVA,amazon,2019-03-12 21:20:00,office-products/490930011,Office Products,69.0,office-products/1069564,Office Products,219.0,2019-03-01,2019-03-12
1,B00006IBVA,amazon,2019-03-13 19:38:52,office-products/490930011,Binder Index Dividers,113.0,office-products/1069564,Binder Accessories,345.0,2019-03-01,2019-03-13
2,B00006IBVA,amazon,2019-03-14 22:14:30,office-products/490930011,Binder Index Dividers,93.0,office-products/1069564,Binder Accessories,297.0,2019-03-01,2019-03-14
3,B00006IBVA,amazon,2019-03-15 21:40:11,office-products/490930011,Binder Index Dividers,121.0,office-products/1069564,Binder Accessories,398.0,2019-03-01,2019-03-15
4,B00006IBVA,amazon,2019-03-16 22:34:32,office-products/490930011,Office Products,118.0,office-products/1069564,Office Products,393.0,2019-03-01,2019-03-16
...,...,...,...,...,...,...,...,...,...,...,...
259,B00006IBVA,amazon,2019-11-26 20:29:21,office-products/490930011,Binder Index Dividers,31.0,office-products,Office Products,6691.0,2019-11-01,2019-11-26
260,B00006IBVA,amazon,2019-11-27 19:23:54,office-products/490930011,Binder Index Dividers,29.0,office-products,Office Products,6486.0,2019-11-01,2019-11-27
261,B00006IBVA,amazon,2019-11-28 20:37:04,office-products/490930011,Binder Index Dividers,26.0,office-products,Office Products,6701.0,2019-11-01,2019-11-28
262,B00006IBVA,amazon,2019-11-29 20:08:32,office-products/490930011,Binder Index Dividers,23.0,office-products,Office Products,7250.0,2019-11-01,2019-11-29


## Aggregate Daily Features

### Try aggregate one feature first

In [27]:
# stars_recent_10: the average star rating of the most recent 10 reviews
for i in tqdm(avery_sales_rank_dict):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict[i].loc[j, "stars_recent_10"] = \
                avery_review_dict[i][avery_review_dict[i]['new_date'] <= \
                                     avery_sales_rank_dict[i].loc[j, 'new_date']].tail(10)['stars'].mean()

100%|██████████| 278/278 [01:23<00:00,  3.33it/s]


In [28]:
avery_sales_rank_dict['B00006IBVA'].head()

,id,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date,stars_recent_10
0,B00006IBVA,amazon,2019-03-12 21:20:00,office-products/490930011,Office Products,69.0,office-products/1069564,Office Products,219.0,2019-03-01,2019-03-12,5.0
1,B00006IBVA,amazon,2019-03-13 19:38:52,office-products/490930011,Binder Index Dividers,113.0,office-products/1069564,Binder Accessories,345.0,2019-03-01,2019-03-13,5.0
2,B00006IBVA,amazon,2019-03-14 22:14:30,office-products/490930011,Binder Index Dividers,93.0,office-products/1069564,Binder Accessories,297.0,2019-03-01,2019-03-14,5.0
3,B00006IBVA,amazon,2019-03-15 21:40:11,office-products/490930011,Binder Index Dividers,121.0,office-products/1069564,Binder Accessories,398.0,2019-03-01,2019-03-15,5.0
4,B00006IBVA,amazon,2019-03-16 22:34:32,office-products/490930011,Office Products,118.0,office-products/1069564,Office Products,393.0,2019-03-01,2019-03-16,5.0


In [29]:
for i in tqdm(avery_sales_rank_dict):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict[i].loc[j, 'stars_recent_oneweek'] = avery_review_dict[i][(avery_review_dict[i]['new_date'] \
                    <= avery_sales_rank_dict[i].loc[j, 'new_date']) & (avery_review_dict[i]['new_date'] >= \
                     (avery_sales_rank_dict[i].loc[j,'new_date'] - timedelta(days = 6)))]['stars'].mean()
                
                avery_sales_rank_dict[i].loc[j, 'stars_recent_onequarter'] = avery_review_dict[i][(avery_review_dict[i]['new_date'] \
                    <= avery_sales_rank_dict[i].loc[j, 'new_date']) & (avery_review_dict[i]['new_date'] >= \
                    (avery_sales_rank_dict[i].loc[j,'new_date'] - timedelta(days = 89)))]['stars'].mean()
                
                avery_sales_rank_dict[i].loc[j, 'stars_avg'] = avery_review_dict[i][avery_review_dict[i]['new_date'] \
                    <= avery_sales_rank_dict[i].loc[j, 'new_date']]['stars'].mean()
                
                avery_sales_rank_dict[i].loc[j, 'review_acc'] = avery_review_dict[i][avery_review_dict[i]['new_date'] \
                    <= avery_sales_rank_dict[i].loc[j, 'new_date']]['new_date'].count()
                
                avery_sales_rank_dict[i].loc[j, 'reviewnum_oneweek'] = avery_review_dict[i][(avery_review_dict[i]['new_date'] \
                    <= avery_sales_rank_dict[i].loc[j, 'new_date']) & (avery_review_dict[i]['new_date'] >= \
                    (avery_sales_rank_dict[i].loc[j,'new_date'] - timedelta(days = 6)))]['new_date'].count()
                
                avery_sales_rank_dict[i].loc[j, 'reviewnum_onequarter'] = avery_review_dict[i][(avery_review_dict[i]['new_date'] \
                    <= avery_sales_rank_dict[i].loc[j, 'new_date']) & (avery_review_dict[i]['new_date'] >= \
                    (avery_sales_rank_dict[i].loc[j,'new_date'] - timedelta(days = 89)))]['new_date'].count()
                
                avery_sales_rank_dict[i].loc[j, 'author_acc'] = avery_review_dict[i][(avery_review_dict[i]['new_date'] \
                    <= avery_sales_rank_dict[i].loc[j,'new_date']) & (avery_review_dict[i]['author'].str.contains('Customer') == False) \
                    & (avery_review_dict[i]['author'].str.contains('customer') == False)]['new_date'].count() / \
                    avery_review_dict[i][avery_review_dict[i]['new_date'] <= avery_sales_rank_dict[i].loc[j,'new_date']]['new_date'].count()
                
                avery_sales_rank_dict[i].loc[j, 'num_1_oneweek'] = avery_review_dict[i][(avery_review_dict[i]['new_date'] \
                    <= avery_sales_rank_dict[i].loc[j, 'new_date']) & (avery_review_dict[i]['new_date'] >= \
                    (avery_sales_rank_dict[i].loc[j,'new_date'] - timedelta(days = 6))) & \
                    (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                
                avery_sales_rank_dict[i].loc[j, 'num_5_oneweek'] = avery_review_dict[i][(avery_review_dict[i]['new_date'] <= \
                    avery_sales_rank_dict[i].loc[j, 'new_date']) & (avery_review_dict[i]['new_date'] >= \
                    (avery_sales_rank_dict[i].loc[j,'new_date'] - timedelta(days = 6))) & \
                    (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                
                avery_sales_rank_dict[i].loc[j, 'num_1_onequarter'] = avery_review_dict[i][(avery_review_dict[i]['new_date'] \
                    <= avery_sales_rank_dict[i].loc[j, 'new_date']) & (avery_review_dict[i]['new_date'] >= \
                    (avery_sales_rank_dict[i].loc[j,'new_date'] - timedelta(days = 89))) & \
                    (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                
                avery_sales_rank_dict[i].loc[j, 'num_5_onequarter'] = avery_review_dict[i][(avery_review_dict[i]['new_date'] <= \
                    avery_sales_rank_dict[i].loc[j, 'new_date']) & (avery_review_dict[i]['new_date'] >= \
                    (avery_sales_rank_dict[i].loc[j,'new_date'] - timedelta(days = 89))) & \
                    (avery_review_dict[i]['stars'] == 5)]['new_date'].count()

100%|██████████| 278/278 [21:12<00:00,  4.58s/it]


In [34]:
avery_sales_rank_dict['B00006IBVA'].head(30)

,id,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,...,reviewnum_onequarter,author_acc,num_1_oneweek,num_5_oneweek,num_1_onequarter,num_5_onequarter,avg_word_count,avg_word_count_10,verified_acc,verified_acc_oneweek
0,B00006IBVA,amazon,2019-03-12 21:20:00,office-products/490930011,Office Products,69.0,office-products/1069564,Office Products,219.0,2019-03-01,...,2.0,0.934426,0.0,1.0,0.0,2.0,15.680328,10.1,0.877049,1.0
1,B00006IBVA,amazon,2019-03-13 19:38:52,office-products/490930011,Binder Index Dividers,113.0,office-products/1069564,Binder Accessories,345.0,2019-03-01,...,3.0,0.934959,0.0,2.0,0.0,3.0,15.569106,8.5,0.878049,1.0
2,B00006IBVA,amazon,2019-03-14 22:14:30,office-products/490930011,Binder Index Dividers,93.0,office-products/1069564,Binder Accessories,297.0,2019-03-01,...,3.0,0.934959,0.0,2.0,0.0,3.0,15.569106,8.5,0.878049,1.0
3,B00006IBVA,amazon,2019-03-15 21:40:11,office-products/490930011,Binder Index Dividers,121.0,office-products/1069564,Binder Accessories,398.0,2019-03-01,...,3.0,0.934959,0.0,1.0,0.0,3.0,15.569106,8.5,0.878049,1.0
4,B00006IBVA,amazon,2019-03-16 22:34:32,office-products/490930011,Office Products,118.0,office-products/1069564,Office Products,393.0,2019-03-01,...,3.0,0.934959,0.0,1.0,0.0,3.0,15.569106,8.5,0.878049,1.0
5,B00006IBVA,amazon,2019-03-17 19:20:08,office-products/490930011,Binder Index Dividers,108.0,office-products/1069564,Binder Accessories,361.0,2019-03-01,...,3.0,0.934959,0.0,1.0,0.0,3.0,15.569106,8.5,0.878049,1.0
6,B00006IBVA,amazon,2019-03-18 21:14:06,office-products/490930011,Binder Index Dividers,94.0,office-products/1069564,Binder Accessories,292.0,2019-03-01,...,3.0,0.934959,0.0,1.0,0.0,3.0,15.569106,8.5,0.878049,1.0
7,B00006IBVA,amazon,2019-03-19 19:45:27,office-products/490930011,Binder Index Dividers,121.0,office-products/1069564,Binder Accessories,378.0,2019-03-01,...,3.0,0.934959,0.0,1.0,0.0,3.0,15.569106,8.5,0.878049,1.0
8,B00006IBVA,amazon,2019-03-20 20:54:54,office-products/490930011,Binder Index Dividers,128.0,office-products/1069564,Binder Accessories,401.0,2019-03-01,...,3.0,0.934959,0.0,0.0,0.0,3.0,15.569106,8.5,0.878049,NaN
9,B00006IBVA,amazon,2019-03-21 21:33:15,office-products/490930011,Binder Index Dividers,149.0,office-products/1069564,Binder Accessories,486.0,2019-03-01,...,3.0,0.934959,0.0,0.0,0.0,3.0,15.569106,8.5,0.878049,NaN


In [31]:
# Aggregate Word Count Features
for i in tqdm(avery_sales_rank_dict):
#     try:
        if i in avery_review_dict:
            for j in avery_sales_rank_dict[i].index.unique():
                if type(avery_review_dict[i]) != pd.core.series.Series:
                    wc_bin = avery_review_dict[i][avery_review_dict[i]['new_date'] <= \
                        avery_sales_rank_dict[i].loc[j, 'new_date']]['text'].tolist()
                    wc_bin_str = ''.join(map(str, wc_bin))
                    if len(wc_bin) > 0:
                        avery_sales_rank_dict[i].loc[j, 'avg_word_count'] = len(wc_bin_str.split())/len(wc_bin)
                    else: 
                        avery_sales_rank_dict[i].loc[j, 'avg_word_count'] = 0
                    
                    wc2_bin = avery_review_dict[i][avery_review_dict[i]['new_date'] <= \
                        avery_sales_rank_dict[i].loc[j, 'new_date']].tail(10)['text'].tolist()
                    wc2_bin_str = ''.join(map(str, wc2_bin))
                    if len(wc2_bin) > 0:
                        avery_sales_rank_dict[i].loc[j, 'avg_word_count_10'] = len(wc2_bin_str.split())/len(wc2_bin)
                    else:
                        avery_sales_rank_dict[i].loc[j, 'avg_word_count_10'] = 0
#     except:
#         continue  

100%|██████████| 278/278 [02:47<00:00,  1.66it/s]


In [32]:
# Aggregate reviewer features
for i in tqdm(avery_sales_rank_dict):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict[i].loc[j, 'verified_acc'] = \
                    (avery_review_dict[i][(avery_review_dict[i]['new_date'] <= avery_sales_rank_dict[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['verified'] == True)]['new_date'].count()) / \
                    (avery_review_dict[i][avery_review_dict[i]['new_date'] <= \
                                        avery_sales_rank_dict[i].loc[j, 'new_date']]['new_date'].count()) 
            
                avery_sales_rank_dict[i].loc[j, 'verified_acc_oneweek'] = \
                    (avery_review_dict[i][(avery_review_dict[i]['new_date'] <= avery_sales_rank_dict[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict[i].loc[j,'new_date'] - \
                    timedelta(days = 6))) & (avery_review_dict[i]['verified'] == True)]['new_date'].count()) / \
                    (avery_review_dict[i][(avery_review_dict[i]['new_date'] <= avery_sales_rank_dict[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict[i].loc[j,'new_date'] - \
                                                      timedelta(days = 6)))]['new_date'].count()) 
       

 50%|█████     | 139/278 [03:31<03:32,  1.53s/it]/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.
100%|██████████| 278/278 [05:20<00:00,  1.15s/it]


In [ ]:
# # aggregate price feature
# for i in tqdm(avery_sales_rank_dict):
#     if i in avery_review_dict:
#         for j in avery_sales_rank_dict[i].index.unique():
#             if type(avery_price_dict[i]) != pd.core.series.Series:
#                 avery_sales_rank_dict[i].loc[j, 'avg_price'] = \ 
#                 avery_price_dict[i][['price', 'month']].groupby('month', as_index = False).mean()['price']

In [33]:
# create monthly metrics dictionary
monthly_metrics_dict = {}
for i in tqdm(avery_sales_rank_dict):
    if i in avery_review_dict: 
        if type(avery_review_dict[i]) != pd.core.series.Series:
            monthly_metrics_dict[i] = pd.DataFrame()

100%|██████████| 278/278 [00:02<00:00, 111.89it/s] 


In [35]:
avery_sales_rank_dict['B00006IBVA'].head()

,id,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,...,reviewnum_onequarter,author_acc,num_1_oneweek,num_5_oneweek,num_1_onequarter,num_5_onequarter,avg_word_count,avg_word_count_10,verified_acc,verified_acc_oneweek
0,B00006IBVA,amazon,2019-03-12 21:20:00,office-products/490930011,Office Products,69.0,office-products/1069564,Office Products,219.0,2019-03-01,...,2.0,0.934426,0.0,1.0,0.0,2.0,15.680328,10.1,0.877049,1.0
1,B00006IBVA,amazon,2019-03-13 19:38:52,office-products/490930011,Binder Index Dividers,113.0,office-products/1069564,Binder Accessories,345.0,2019-03-01,...,3.0,0.934959,0.0,2.0,0.0,3.0,15.569106,8.5,0.878049,1.0
2,B00006IBVA,amazon,2019-03-14 22:14:30,office-products/490930011,Binder Index Dividers,93.0,office-products/1069564,Binder Accessories,297.0,2019-03-01,...,3.0,0.934959,0.0,2.0,0.0,3.0,15.569106,8.5,0.878049,1.0
3,B00006IBVA,amazon,2019-03-15 21:40:11,office-products/490930011,Binder Index Dividers,121.0,office-products/1069564,Binder Accessories,398.0,2019-03-01,...,3.0,0.934959,0.0,1.0,0.0,3.0,15.569106,8.5,0.878049,1.0
4,B00006IBVA,amazon,2019-03-16 22:34:32,office-products/490930011,Office Products,118.0,office-products/1069564,Office Products,393.0,2019-03-01,...,3.0,0.934959,0.0,1.0,0.0,3.0,15.569106,8.5,0.878049,1.0


In [36]:
for i in tqdm(monthly_metrics_dict):
    try: 
        df_bin = avery_sales_rank_dict[i][['category_rank1', 'stars_recent_oneweek', 'stars_recent_onequarter',
           'stars_avg', 'review_acc', 'reviewnum_oneweek', 'reviewnum_onequarter',
           'author_acc', 'num_1_oneweek', 'num_5_oneweek', 'num_1_onequarter',
           'num_5_onequarter', 'avg_word_count', 'avg_word_count_10',
           'verified_acc', 'verified_acc_oneweek', 'month']].groupby('month', as_index = False).mean()
        df_bin.rename(columns = {'category_rank1': 'avg_monthly_rank'}, inplace = True)
        monthly_metrics_dict[i] = df_bin
        monthly_metrics_dict[i]['product'] = i
        monthly_metrics_dict[i]['log_rank'] = monthly_metrics_dict[i]['avg_monthly_rank'].map(lambda x: math.log(x))
        monthly_metrics_dict[i]['delta_rank'] = monthly_metrics_dict[i]['log_rank'].diff()
        monthly_metrics_dict[i]['avg_price'] = avery_price_dict[i][['price', 'month']].groupby('month',
                                                                as_index = False).mean()['price']
        
    except:
        print("error", i)
        continue

100%|██████████| 202/202 [00:03<00:00, 66.32it/s]


In [ ]:
monthly_metrics_dict['B00004Z5QO'][['product', 'month', 'log_rank',
       'delta_rank', 'avg_monthly_rank', 'stars_recent_oneweek',
       'stars_recent_onequarter', 'stars_avg', 'review_acc',
       'reviewnum_oneweek', 'reviewnum_onequarter', 'author_acc',
       'num_1_oneweek', 'num_5_oneweek', 'num_1_onequarter',
       'num_5_onequarter', 'avg_word_count', 'avg_word_count_10',
       'verified_acc', 'verified_acc_oneweek', 'avg_price']]

In [37]:
reg_table = pd.DataFrame()
for i in monthly_metrics_dict:
    if type(monthly_metrics_dict[i]) != pd.core.series.Series:
        df_bin = monthly_metrics_dict[i][['product', 'month', 'log_rank',
       'delta_rank', 'avg_monthly_rank', 'stars_recent_oneweek',
       'stars_recent_onequarter', 'stars_avg', 'review_acc',
       'reviewnum_oneweek', 'reviewnum_onequarter', 'author_acc',
       'num_1_oneweek', 'num_5_oneweek', 'num_1_onequarter',
       'num_5_onequarter', 'avg_word_count', 'avg_word_count_10',
       'verified_acc', 'verified_acc_oneweek', 'avg_price']]
    else:
        df_bin = monthly_metrics_dict[i].to_frame().T
    reg_table = pd.concat([reg_table, df_bin], ignore_index = True)

In [38]:
reg_table.head(50)

,product,month,log_rank,delta_rank,avg_monthly_rank,stars_recent_oneweek,stars_recent_onequarter,stars_avg,review_acc,reviewnum_oneweek,...,author_acc,num_1_oneweek,num_5_oneweek,num_1_onequarter,num_5_onequarter,avg_word_count,avg_word_count_10,verified_acc,verified_acc_oneweek,avg_price
0,B00006IBVA,2019-03-01,4.789989,NaN,120.300000,5.000000,5.000000,4.520129,122.950000,0.500000,...,0.934933,0.0,0.500000,0.0,2.950000,15.574667,8.580000,0.877999,1.000000,NaN
1,B00006IBVA,2019-04-01,4.867791,0.077802,130.033333,NaN,5.000000,4.520325,123.000000,0.000000,...,0.934959,0.0,0.000000,0.0,3.000000,15.569106,8.500000,0.878049,NaN,NaN
2,B00006IBVA,2019-05-01,4.970276,0.102485,144.066667,NaN,5.000000,4.520325,123.000000,0.000000,...,0.934959,0.0,0.000000,0.0,2.483871,15.569106,8.500000,0.878049,NaN,NaN
3,B00006IBVA,2019-06-01,5.022783,0.052507,151.833333,NaN,5.000000,4.520325,123.000000,0.000000,...,0.934959,0.0,0.000000,0.0,0.500000,15.569106,8.500000,0.878049,NaN,3.080000
4,B00006IBVA,2019-07-01,5.053777,0.030994,156.612903,NaN,NaN,4.520325,123.000000,0.000000,...,0.934959,0.0,0.000000,0.0,0.000000,15.569106,8.500000,0.878049,NaN,NaN
5,B00006IBVA,2019-08-01,4.048132,-1.005645,57.290323,5.000000,5.000000,4.520450,123.032258,0.032258,...,0.934976,0.0,0.032258,0.0,0.032258,15.565055,8.496774,0.878081,1.000000,2.390323
6,B00006IBVA,2019-09-01,3.599137,-0.448995,36.566667,4.656944,4.776190,4.522705,126.800000,1.600000,...,0.933802,0.0,0.900000,0.0,2.666667,15.485865,11.946667,0.881655,1.000000,2.790000
7,B00006IBVA,2019-10-01,4.222350,0.623213,68.193548,5.000000,4.500000,4.519084,131.000000,0.225806,...,0.923664,0.0,0.225806,0.0,5.000000,15.541985,12.600000,0.885496,1.000000,2.447419
8,B00006IBVA,2019-11-01,3.610017,-0.612333,36.966667,NaN,4.495238,4.519084,131.000000,0.000000,...,0.923664,0.0,0.000000,0.0,4.933333,15.541985,12.600000,0.885496,NaN,1.455517
9,B00006IBVG,2019-03-01,2.884801,NaN,17.900000,3.500000,4.437500,4.727713,211.350000,0.700000,...,0.929027,0.0,0.350000,0.0,3.600000,17.461410,12.820000,0.914832,1.000000,6.195806


In [39]:
reg_table.to_csv("Avery_Regression_Table_490930011.csv")